In [2]:
import pandas as pd
import numpy as np
from itertools import product
import math


In [3]:
activations_list = [[1,-1,1,-1], [1,-1,1,-1], [-1,1,-1,-1], [-1,1,1,1], [1,1,1,-1], [-1,-1,1,1], [1,-1,-1,1], [1,-1,1,1]]
activations_matrix = pd.DataFrame(activations_list, columns=['sigma_0', 'sigma_1', 'sigma_2', 'sigma_3'])
activations_matrix.values

array([[ 1, -1,  1, -1],
       [ 1, -1,  1, -1],
       [-1,  1, -1, -1],
       [-1,  1,  1,  1],
       [ 1,  1,  1, -1],
       [-1, -1,  1,  1],
       [ 1, -1, -1,  1],
       [ 1, -1,  1,  1]], dtype=int64)

In [6]:
neurons_corr = activations_matrix.corr().values
neurons_corr

array([[ 1.        , -0.46666667,  0.1490712 , -0.25819889],
       [-0.46666667,  1.        , -0.1490712 , -0.25819889],
       [ 0.1490712 , -0.1490712 ,  1.        ,  0.        ],
       [-0.25819889, -0.25819889,  0.        ,  1.        ]])

In [7]:
neurons_average = activations_matrix.mean().values
neurons_average

array([ 0.25, -0.25,  0.5 ,  0.  ])

According to Hinton's "A Practical Guide to Training Restricted Boltzmann Machines"

Initialize J matrix to zero mean and 0.01 sd. The diagonal will be 0.

Initizlize bias b_i  of visible unit i to log[pi/(1−pi)]

In [8]:
np.random.seed(3)
J = np.random.normal(loc=0.0, scale=.01, size=neurons_corr.shape)
#np.fill_diagonal(J,0)
lind = np.tril_indices(4)
J[lind]=0
J

array([[ 0.        ,  0.0043651 ,  0.00096497, -0.01863493],
       [ 0.        ,  0.        , -0.00082741, -0.00627001],
       [ 0.        ,  0.        ,  0.        ,  0.00884622],
       [ 0.        ,  0.        ,  0.        ,  0.        ]])

In [9]:
H = activations_matrix.sum()/activations_matrix.shape[0]

In [10]:
J

array([[ 0.        ,  0.0043651 ,  0.00096497, -0.01863493],
       [ 0.        ,  0.        , -0.00082741, -0.00627001],
       [ 0.        ,  0.        ,  0.        ,  0.00884622],
       [ 0.        ,  0.        ,  0.        ,  0.        ]])

In [11]:
H

sigma_0    0.25
sigma_1   -0.25
sigma_2    0.50
sigma_3    0.00
dtype: float64

In [12]:
spin_values = [-1,1]

# Calcular las probabilidades
# Se pueden iterar todos los valores antes y guardarlos en un diccionario
prob_dict = {}
for sigma in product(spin_values, repeat=activations_matrix.shape[1]):
    prob_dict[sigma] = 0
    for i in range(0,activations_matrix.shape[1]):
        for j in range(i+1,activations_matrix.shape[1]):
            prob_dict[sigma] += sigma[i]*sigma[j]*J[i][j]
            
    for i in range(0,activations_matrix.shape[1]):
        prob_dict[sigma] += sigma[i]*H[i]
    
    prob_dict[sigma] = math.exp(prob_dict[sigma])
            
Z = sum(prob_dict.values())

for sigma in prob_dict.keys():
    prob_dict[sigma] = prob_dict[sigma]/Z


In [13]:
corr_model_exp = np.zeros(J.shape)
for sigma in product(spin_values, repeat=activations_matrix.shape[1]):
    for i in range(0,activations_matrix.shape[1]):
        for j in range(i+1,activations_matrix.shape[1]):
            print(i,j)
            corr_model_exp[i][j] += sigma[i]*sigma[j]*prob_dict[sigma]
    break
corr_model_exp

0 1
0 2
0 3
1 2
1 3
2 3


array([[0.        , 0.03123291, 0.03123291, 0.03123291],
       [0.        , 0.        , 0.03123291, 0.03123291],
       [0.        , 0.        , 0.        , 0.03123291],
       [0.        , 0.        , 0.        , 0.        ]])

In [14]:
mean_model_exp = np.zeros(H.shape)
for sigma in product(spin_values, repeat=activations_matrix.shape[1]):
    for i in range(0,activations_matrix.shape[1]):
        mean_model_exp[i] += sigma[i]*prob_dict[sigma]
mean_model_exp

array([ 0.24423279, -0.24426888,  0.4624399 ,  0.00106991])

In [17]:
class IsingModel:
    def train(self, data ,lr=10**-4):
        np.random.seed(3)
        
        activations_matrix = pd.DataFrame(activations_list, columns=['sigma_0', 'sigma_1', 'sigma_2', 'sigma_3'])
        # Calculate the true data expectations
        spins_correlation = activations_matrix.corr().values
        lind = np.tril_indices_from(spins_correlation)
        spins_correlation[lind]=0
        spins_average = activations_matrix.mean().values

        # Calculate the initial J matrix. Fill only the upper diagonal
        # Filling with Normal(0, 0.01)
        J = np.random.normal(loc=0.0, scale=.01, size=spins_correlation.shape)
        lind = np.tril_indices_from(J)
        J[lind]=0
                
        # Calculate the initial H vector, each entry proportional to its probability (according to Hinton)
        H = activations_matrix.sum()/activations_matrix.shape[0]
        
        #totalParamVariation = math.inf
        stopCondition = 0.5
        
        epoch = 1
        
        while totalParamVariation > stopCondition and epoch < 1000:
            # Calculate P(sigma) for every possible combination in the model
            spin_values = [-1,1]
            prob_dict = {}
            for sigma in product(spin_values, repeat=activations_matrix.shape[1]):
                prob_dict[sigma] = 0
                for i in range(0,activations_matrix.shape[1]):
                    for j in range(i+1,activations_matrix.shape[1]):
                        prob_dict[sigma] += sigma[i]*sigma[j]*J[i][j]

                for i in range(0,activations_matrix.shape[1]):
                    prob_dict[sigma] += sigma[i]*H[i]

                prob_dict[sigma] = math.exp(prob_dict[sigma])

            # Calculate the partition function
            Z = sum(prob_dict.values())

            # Normalize dividing by the partition function
            for sigma in prob_dict.keys():
                prob_dict[sigma] = prob_dict[sigma]/Z

            # Calculate the expectation of the correlations
            corr_model_exp = np.zeros(J.shape)
            for sigma in product(spin_values, repeat=activations_matrix.shape[1]):
                for i in range(0,activations_matrix.shape[1]):
                    for j in range(i+1,activations_matrix.shape[1]):
                        corr_model_exp[i][j] += sigma[i]*sigma[j]*prob_dict[sigma]

            # Calculate the expectation of the averages
            mean_model_exp = np.zeros(H.shape)
            for sigma in product(spin_values, repeat=activations_matrix.shape[1]):
                for i in range(0,activations_matrix.shape[1]):
                    mean_model_exp[i] += sigma[i]*prob_dict[sigma]


            # Calculate the step size for every J_{ij}
            stepJ = lr * (spins_correlation - corr_model_exp)
            # Calculate the variation of J for the termination condition
            deltaJ = abs(J - stepJ).sum().sum()
            # Take the step
            J = J + stepJ

            # Calculate the step size for every H_{i}
            stepH = lr * (spins_average - mean_model_exp)
            # Calculate the variation of J for the termination condition
            deltaH = abs(H - stepH).sum()
            # Take the step
            H = H + stepH
            totalParamVariation = 0
            totalParamVariation = deltaH + deltaJ
            
            print('Epoch', epoch, 'TotalParamVariation', totalParamVariation)
            
            epoch += 1

        print(H)
        print(spins_correlation)
        print(J)

In [16]:
activations_list = [[1,-1,1,-1], [1,-1,1,-1], [-1,1,-1,-1], [-1,1,1,1], [1,1,1,-1], [-1,-1,1,1], [1,-1,-1,1], [1,-1,1,1]]


ising = IsingModel()
ising.train(activations_list)

Epoch 1 TotalParamVariation 1.0398892989166255
Epoch 2 TotalParamVariation 1.0399086504799808
Epoch 3 TotalParamVariation 1.0399282111519164
Epoch 4 TotalParamVariation 1.0399477669514146
Epoch 5 TotalParamVariation 1.0399673178796125
Epoch 6 TotalParamVariation 1.039986863937648
Epoch 7 TotalParamVariation 1.0400064051266569
Epoch 8 TotalParamVariation 1.0400259414477755
Epoch 9 TotalParamVariation 1.0400454729021407
Epoch 10 TotalParamVariation 1.0400649994908884
Epoch 11 TotalParamVariation 1.0400845212151542
Epoch 12 TotalParamVariation 1.040104038076074
Epoch 13 TotalParamVariation 1.0401235500747825
Epoch 14 TotalParamVariation 1.0401430572124157
Epoch 15 TotalParamVariation 1.0401625594901083
Epoch 16 TotalParamVariation 1.040182056908995
Epoch 17 TotalParamVariation 1.0402015494702108
Epoch 18 TotalParamVariation 1.0402210371748897
Epoch 19 TotalParamVariation 1.0402405200241656
Epoch 20 TotalParamVariation 1.0402599980191731
Epoch 21 TotalParamVariation 1.040279471161045
Epoch

Epoch 314 TotalParamVariation 1.062299634144681
Epoch 315 TotalParamVariation 1.0623973990452154
Epoch 316 TotalParamVariation 1.06249515151633
Epoch 317 TotalParamVariation 1.062592891560267
Epoch 318 TotalParamVariation 1.0626906191792689
Epoch 319 TotalParamVariation 1.0627883343755773
Epoch 320 TotalParamVariation 1.0628860371514326
Epoch 321 TotalParamVariation 1.0629837275090765
Epoch 322 TotalParamVariation 1.0630814054507494
Epoch 323 TotalParamVariation 1.0631790709786901
Epoch 324 TotalParamVariation 1.0632767240951404
Epoch 325 TotalParamVariation 1.0633743648023375
Epoch 326 TotalParamVariation 1.0634719931025216
Epoch 327 TotalParamVariation 1.0635696089979312
Epoch 328 TotalParamVariation 1.063667212490803
Epoch 329 TotalParamVariation 1.063764803583376
Epoch 330 TotalParamVariation 1.0638623822778872
Epoch 331 TotalParamVariation 1.0639599485765732
Epoch 332 TotalParamVariation 1.064057502481671
Epoch 333 TotalParamVariation 1.0641550439954162
Epoch 334 TotalParamVariati

Epoch 642 TotalParamVariation 1.0937128395275446
Epoch 643 TotalParamVariation 1.093806645563572
Epoch 644 TotalParamVariation 1.093900439884865
Epoch 645 TotalParamVariation 1.0939942224935395
Epoch 646 TotalParamVariation 1.0940879933917107
Epoch 647 TotalParamVariation 1.0941817525814947
Epoch 648 TotalParamVariation 1.0942755000650055
Epoch 649 TotalParamVariation 1.0943692358443577
Epoch 650 TotalParamVariation 1.094462959921665
Epoch 651 TotalParamVariation 1.0945566722990416
Epoch 652 TotalParamVariation 1.0946503729786006
Epoch 653 TotalParamVariation 1.094744061962455
Epoch 654 TotalParamVariation 1.094837739252717
Epoch 655 TotalParamVariation 1.0949314048514989
Epoch 656 TotalParamVariation 1.0950250587609127
Epoch 657 TotalParamVariation 1.0951187009830694
Epoch 658 TotalParamVariation 1.09521233152008
Epoch 659 TotalParamVariation 1.0953059503740556
Epoch 660 TotalParamVariation 1.095399557547106
Epoch 661 TotalParamVariation 1.0954931530413412
Epoch 662 TotalParamVariatio

Epoch 933 TotalParamVariation 1.1205245838433096
Epoch 934 TotalParamVariation 1.120615068652147
Epoch 935 TotalParamVariation 1.1207055423462489
Epoch 936 TotalParamVariation 1.1207960049276227
Epoch 937 TotalParamVariation 1.1208864563982768
Epoch 938 TotalParamVariation 1.120976896760218
Epoch 939 TotalParamVariation 1.1210673260154542
Epoch 940 TotalParamVariation 1.1211577441659912
Epoch 941 TotalParamVariation 1.1212481512138361
Epoch 942 TotalParamVariation 1.1213385471609942
Epoch 943 TotalParamVariation 1.1214289320094715
Epoch 944 TotalParamVariation 1.121519305761273
Epoch 945 TotalParamVariation 1.1216096684184031
Epoch 946 TotalParamVariation 1.1217000199828673
Epoch 947 TotalParamVariation 1.1217903604566688
Epoch 948 TotalParamVariation 1.121880689841812
Epoch 949 TotalParamVariation 1.1219710081402998
Epoch 950 TotalParamVariation 1.122061315354136
Epoch 951 TotalParamVariation 1.1221516114853223
Epoch 952 TotalParamVariation 1.1222418965358616
Epoch 953 TotalParamVaria